### 0. data ingestion

In [1]:
import data_ingestion
import os
import pandas as pd
from typing import Any
import re
import numpy as np

# Define constants for dataset details and paths
DATASET_ORIGIN = 'rashikrahmanpritom'
DATASET_NAME = 'data-science-job-posting-on-glassdoor'
DOWNLOAD_PATH = 'data'

# Call the function to download and unzip the dataset
# data_ingestion.download_and_unzip_kaggle_dataset(DATASET_ORIGIN, DATASET_NAME, DOWNLOAD_PATH)

csv_file_path = os.path.join(DOWNLOAD_PATH, 'Uncleaned_DS_Jobs.csv')
df = data_ingestion.load_csv(csv_file_path)

CSV file 'data\Uncleaned_DS_Jobs.csv' loaded successfully.


---

### 1. data cleaning

In [2]:
def normalize_column_names(df):
    """
    Convert DataFrame column names to lowercase and replace spaces with underscores.
    This function might break YAGNI rule but it's defined in case more complex rules are needed for normalization.
    
    Parameters:
        df (pd.DataFrame): The DataFrame with original column names.
        
    Returns:
        pd.DataFrame: The DataFrame with normalized column names.
    """
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

df = normalize_column_names(df)

In [3]:
df = df.drop('index', axis=1)
df = df.drop('job_description', axis=1)

In [4]:
# Create a new column with the text proceeding the newline character
# df[['company_name', 'company_star_rating']] = df['company_name'].str.split('\n', expand=True)
df['company_name'] = df['company_name'].str.split('\n').str[0]

In [5]:
df['salary_estimate'] = df['salary_estimate'].str.extract(r'(\$\d+K-\$\d+K)')

In [6]:
def replace_vals_in_cols(df: pd.DataFrame, cols: list[str], old_val: Any, new_val: Any) -> pd.DataFrame:
    """
    Replace occurrences of `old_val` with `new_val` in specified columns of the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        cols (list[str]): A list of column names in which to perform the replacement.
        old_val (Any): The value to be replaced.
        new_val (Any): The value to replace `old_val` with.

    Returns:
        pd.DataFrame: The DataFrame with the specified values replaced.
    """
    for col in cols:
        df[col] = df[col].replace(old_val, new_val)
    
    return df

cols_list = ['rating', 'headquarters', 'size', 'founded', 'type_of_ownership', 'industry', 'sector', 'revenue', 'competitors']
df = replace_vals_in_cols(df, cols_list, ['-1', -1, -1.0, 'Unknown / Non-Applicable'], pd.NA)

---

### 2. data transformation

In [7]:
def extract_revenue_range(s):
    """Extract the minimum and maximum revenue values from a revenue string.

    This function takes a revenue string in various formats such as:
      - "$X to $Y million/billion (USD)"
      - "$X million/billion to $Y million/billion (USD)"
      - "$X+ million/billion (USD)"
      - "Less than $X million/billion (USD)"
    and extracts the numeric lower and upper revenue bounds, converting them to USD numbers.
    If only one bound is available, the other is set to NaN. If the string is not in a recognized
    format or is missing, both values will be NaN.

    Args:
        s (str or NA): A revenue string representing a range of revenue values or a partial value.

    Returns:
        pandas.Series: A Series with two elements:
            - The first element is the lower bound revenue as a float (in USD), or NaN if unavailable.
            - The second element is the upper bound revenue as a float (in USD), or NaN if unavailable.
    """
    if pd.isna(s):
        return pd.Series([np.nan, np.nan])
    
    # Pattern 1: "$X million/billion to $Y million/billion"
    # e.g., "$500 million to $1 billion (USD)"
    match = re.search(
        r'\$([\d\.]+)\s*(million|billion)\s*to\s*\$([\d\.]+)\s*(million|billion)',
        s, re.IGNORECASE
    )
    if match:
        low, low_unit, high, high_unit = match.groups()
        low_factor = 1e6 if low_unit.lower() == 'million' else 1e9
        high_factor = 1e6 if high_unit.lower() == 'million' else 1e9
        return pd.Series([float(low) * low_factor, float(high) * high_factor])
    
    # Pattern 2: "$X to $Y (unit)"
    # e.g., "$1 to $2 billion (USD)" where the unit is only provided once.
    match = re.search(
        r'\$([\d\.]+)\s*to\s*\$([\d\.]+)\s*(million|billion)',
        s, re.IGNORECASE
    )
    if match:
        low, high, unit = match.groups()
        factor = 1e6 if unit.lower() == 'million' else 1e9
        return pd.Series([float(low) * factor, float(high) * factor])
    
    # Pattern 3: "$X+ (unit)" (only lower bound available)
    # e.g., "$10+ billion (USD)"
    match = re.search(
        r'\$([\d\.]+)\+\s*(million|billion)',
        s, re.IGNORECASE
    )
    if match:
        low, unit = match.groups()
        factor = 1e6 if unit.lower() == 'million' else 1e9
        return pd.Series([float(low) * factor, np.nan])
    
    # Pattern 4: "Less than $X (unit)" (only upper bound available)
    # e.g., "Less than $1 million (USD)"
    match = re.search(
        r'Less than\s*\$([\d\.]+)\s*(million|billion)',
        s, re.IGNORECASE
    )
    if match:
        high, unit = match.groups()
        factor = 1e6 if unit.lower() == 'million' else 1e9
        return pd.Series([0.0, float(high) * factor])
    
    # If no pattern matches, return NaN for both bounds
    return pd.Series([np.nan, np.nan])

# Apply the function to create two new columns: min_revenue and max_revenue
df[['min_revenue', 'max_revenue']] = df['revenue'].apply(extract_revenue_range)

df = df.drop(columns=['revenue'])

In [8]:
df[['salary_min', 'salary_max']] = df['salary_estimate'].str.extract(r'\$(\d+)K-\$(\d+)K')
df['salary_min'] = df['salary_min'].astype(int) * 1000
df['salary_max'] = df['salary_max'].astype(int) * 1000

df = df.drop(columns=['salary_estimate'])

df['location_city'] = df['location'].str.split(',').str[0]
df['location_state'] = df['location'].str.split(',').str[1].str.strip()

df['headquarters_city'] = df['headquarters'].str.split(',').str[0]
df['headquarters_state'] = df['headquarters'].str.split(',').str[1].str.strip()

df = df.drop(columns=['location', 'headquarters'])

df['num_competitors'] = df['competitors'].str.split(',').str.len()

df = df.drop(columns=['competitors'])

In [9]:
df.head()

,job_title,rating,company_name,size,founded,type_of_ownership,industry,sector,min_revenue,max_revenue,salary_min,salary_max,location_city,location_state,headquarters_city,headquarters_state,num_competitors
0,Sr Data Scientist,3.1,Healthfirst,1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,NaN,NaN,137000,171000,New York,NY,New York,NY,3
1,Data Scientist,4.2,ManTech,5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,1.000000e+09,2.000000e+09,137000,171000,Chantilly,VA,Herndon,VA,<NA>
2,Data Scientist,3.8,Analysis Group,1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,1.000000e+08,5.000000e+08,137000,171000,Boston,MA,Boston,MA,<NA>
3,Data Scientist,3.5,INFICON,501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,1.000000e+08,5.000000e+08,137000,171000,Newton,MA,Bad Ragaz,Switzerland,3
4,Data Scientist,2.9,Affinity Solutions,51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,NaN,NaN,137000,171000,New York,NY,New York,NY,3


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   job_title           672 non-null    object 
 1   rating              622 non-null    object 
 2   company_name        672 non-null    object 
 3   size                645 non-null    object 
 4   founded             554 non-null    object 
 5   type_of_ownership   645 non-null    object 
 6   industry            601 non-null    object 
 7   sector              601 non-null    object 
 8   min_revenue         432 non-null    float64
 9   max_revenue         369 non-null    float64
 10  salary_min          672 non-null    int64  
 11  salary_max          672 non-null    int64  
 12  location_city       672 non-null    object 
 13  location_state      649 non-null    object 
 14  headquarters_city   641 non-null    object 
 15  headquarters_state  641 non-null    object 
 16  num_comp